<h4>Libs and housekeeping</h4>

In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smtsa

import pmdarima as pm

from sklearn.metrics import mean_absolute_error, mean_squared_error , mean_absolute_percentage_error

In [6]:
RANDOM_STATE = 24
np.random.seed(RANDOM_STATE)
rnd = np.random.RandomState(RANDOM_STATE)

NUM_SAMPLES = 600

params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 7),
          'axes.labelsize': 'large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'large',
          'savefig.dpi': 75,
          'image.interpolation': 'none',
          'savefig.bbox' : 'tight',
          'lines.linewidth' : 1,
          'legend.numpoints' : 1,
          'scatter.edgecolors': 'b' 
         }

CMAP = plt.cm.brg
plt.rcParams.update(params);
plt.set_cmap(CMAP);

plt.style.use('seaborn-v0_8-darkgrid')

<Figure size 1500x700 with 0 Axes>

<H4>Helper function </H4>

In [25]:
def plotds(xt, hlag = 30, figsize = (15,12)):
    if not isinstance(xt, pd.series):
            xt = pd.series(xt)
    plt.figure(figsize = figsize)

    layout = (2,2)

    # assign Axis
    ax_xt = plt.subplot2grid(layout, (0,0), colspan=2)
    ax_acf = plt.subplot2grid(layout,(1,0))
    ax_pacf = plt.subplot2grid(layout,(1,1))
    
    xt.plot(ax=ax_xt)
    ax_xt.set_title('time Series')

    plot_acf(xt, lags=nlag, ax =ax_acf)
    plot_pacf(xt, lags=nlag, ax =ax_pacf)
    plt.tight_layout()